<a href="https://colab.research.google.com/github/yuji-sgs/m1-reserch/blob/main/MOPSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

## 本ノートブックの目的
[前回作成したノートブック](https://github.com/yuji-sgs/m1-reserch/blob/main/Inverse_Analysis.ipynb)では深層学習モデルの1出力に対する逆解析にしか適応できなかったが、複数出力に対する逆解析を実現するために、KerasのFunctional APIモデルを用いて複数出力の深層学習モデルを作成し、その学習済みモデルを適用して多目的粒子群最適化（MOPSO）アルゴリズムを構築していく。

## 処理フロー
1. 使用するライブラリをインポート
2. データの読み込み・確認
3. データ分割
4. KerasのFunctional APIモデルで学習
5. 学習後の精度評価
6. MOPSO実装